# 0. Data

In [57]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
device = torch.device("cuda")
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.linalg import block_diag # for computation efficiency - Kronecker product with identity matrix and regular matrix

In [58]:
'split data'
'first 21 columns are input variables and last 7 columns are output variables'

train_data = pd.read_csv('./data/SARCOSTst.csv', header=None)[:1000]
test_data = pd.read_csv('./data/SARCOSTrn.csv', header=None)[:100]

X_train, X_valid, Y_train, Y_valid = train_test_split(train_data.iloc[:,:21], train_data.iloc[:,-7:], test_size=0.2)
X_test, Y_test = test_data.iloc[:,:21], test_data.iloc[:,-7:]

X_train = torch.tensor(X_train.values).to(device)
X_valid = torch.tensor(X_valid.values).to(device)
Y_train = torch.tensor(Y_train.values).to(device)
Y_valid = torch.tensor(Y_valid.values).to(device)
X_test = torch.tensor(X_test.values).to(device)
Y_test = torch.tensor(Y_test.values).to(device)

print('train set: ', X_train.shape, Y_train.shape)
print('valid set: ', X_valid.shape, Y_valid.shape)
print('test set: ', X_test.shape, Y_test.shape)

train set:  torch.Size([800, 21]) torch.Size([800, 7])
valid set:  torch.Size([200, 21]) torch.Size([200, 7])
test set:  torch.Size([100, 21]) torch.Size([100, 7])


# 1. Kernel

In [87]:
class LinearKernel:
    """
    standard dot product kernel k(a,b) = a^\top b
    :input: X1 (N*D), X2 (M*D)
    :output: covariance matrix (N*M)
    """
    def __init__(self):
        pass

    def __call__(self, X1, X2):
        # return torch.tensor([[torch.dot(x1, x2) for x1 in X1] for x2 in X2]) 
        return torch.matmul(X1, X2.T) 


In [79]:
class GaussianKernel:
    """
    isotropic Gaussian kernel
    :input: X1 (N*D), X2 (M*D)
    :output: covariance matrix (N*M)
    """
    def __init__(self, sigma_k=1):
        self.sigma_k = sigma_k # isotropic Gaussian kernel variance (Hyperparameter !)

    def __call__(self, X1, X2):
        # return np.exp(-(np.sum(X1**2, axis=1).values.reshape(-1, 1) +
        #                 np.sum(X2**2, axis=1).values.reshape(1, -1) - 2*X1@X2.T) / pow(self.sigma_k, 2))
        return torch.exp(-(torch.sum(X1**2, axis=1).reshape(-1, 1) +
                           torch.sum(X2**2, axis=1).reshape(1, -1) - 2*torch.matmul(X1, X2.T)) / pow(self.sigma_k, 2))


# 2. GP regression

In [100]:
class GP_Regression(nn.Module):
    def __init__(self, X_train, Y_train, K:GaussianKernel(), sigma_n, device):
        super().__init__()
        self.X_train = X_train
        self.Y_train = Y_train
        self.K = K
        self.sigma_n = nn.Parameter(torch.tensor(sigma_n),requires_grad=True)  # noise variance (Hyperparameter)
        self.device = device
        self.D = self.Y_train.shape[1] # output dim (=7)
    
    def fit(self, X_test):
        """
        calculate sufficient statistics
        """
        self.K_X_X = self.K(self.X_train, self.X_train)
        self.K_X_X = block_diag(*[self.K_X_X]*self.D)
        self.K_Xt_X = self.K(X_test, self.X_train)
        self.K_Xt_X = block_diag(*[self.K_Xt_X]*self.D)
        self.K_X_Xt = self.K_Xt_X.T
        self.K_Xt_Xt = self.K(X_test, X_test)
        self.K_Xt_Xt = block_diag(*[self.K_Xt_Xt]*self.D)
        self.Sigma = self.sigma_n * torch.eye(self.D)
        self.I = np.identity(len(self.X_train))
        self.y_concat = self.Y_train.T.stack(level=-1).values

    def __NLL_term_1__(self):
        combined = []
        for d in range(self.D):
            combined.append( 
                (-0.5 * torch.matmul( torch.matmul(self.Y_train, self.K_X_X+torch.kron(self.Sigma,self.I)), self.Y_train.T )).item() 
            )
        return sum(combined)

    def __NLL_term_2__(self):
        return -0.5 * torch.log(torch.det(self.K_X_X + torch.kron(self.Sigma, self.I)))
    
    def predict(self, X_test):
        
        K_X_X, K_X_Xt, K_Xt_X, K_Xt_Xt, Sigma, I, y_concat = self.K_X_X, self.K_X_Xt, self.K_Xt_X, self.K_Xt_Xt, self.Sigma, self.I, self.y_concat

        # predictive mean, covariance, variance
        mean = K_Xt_X @ np.linalg.inv( K_X_X + torch.kron(Sigma,I) ) @ y_concat
        cov = K_Xt_Xt - K_Xt_X @ np.linalg.inv( K_X_X + torch.kron(Sigma,I) ) @ K_X_Xt
        var = np.diag(cov)

        self.predictive_distribution = {'mean': mean, 'cov':cov, 'var':var}
        self.temp = [K_X_X, K_X_Xt, K_Xt_X, K_Xt_Xt, Sigma, I, y_concat]

        return pd.DataFrame(mean.reshape(self.D, -1)).T


SyntaxError: invalid syntax (1023036334.py, line 33)

In [40]:
model = GP_Regression(X_train, Y_train, GaussianKernel())
pred = model.predict(X_test)

mse = mean_squared_error(pred.values, Y_test.values)
print(mse)

C = model.temp[0]
K_X_X = model.temp[1]
K_X_Xt = model.temp[2]
K_Xt_X = model.temp[3]
K_Xt_Xt = model.temp[4]
Sigma = model.temp[5]
I = model.temp[6]
y_concat = model.temp[7]


381.8446973048145


# 3. Main